In [ ]:
from ssl import AlertDescription
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from PIL import Image as im 
# '''le travail demander est de detecter la note de l'étudiant du tableau coché par l'enseignant
# c'est pour cela la méthode idéale est de trouver le carreau cohé par l'enseignant , mais pour des raisons de vérification 
# j'ai utiliser la méthode inverse qui insiste à vérifier les carreaux blanc qui ne sont pas cochés pour mieux de vérification
# pour l'implementation de cette fonctionalité la fonction principale est cv2.matchTemplate
# cette fonction permet à travers des convolutions entre l'image origine et l'image template,cropper de l'image initiale ,
# à déterminer la position prés du template dans cette image '''


# la fonction cropping prend en input l image et les point (xmin,ymin) et (xmax, ymax)
#affin de cropper le carreau qui de size égal à 106 mais pour ne pas avoir des conflits 
#j'ai pris 116 

def cropping (image,min_x , min_y, max_x , max_y,saving_name):
    I = image[min_x-106: max_x, min_y-106:max_y]
    data = im.fromarray(I)
    data.save(saving_name)

#cropping the table of comma 's point 0.00 to 0.75
def cropping_comma(image):
  crooped_image=image[1400:1750,500:1600]
  data=im.fromarray(crooped_image)
  data.save("firgule.png")
  return crooped_image

#cropping the table of numbers from 0 to 20
def cropping_note(image):
  crooped_image=image[800:1400,0:2481]
  return crooped_image

def cropping_comma(image):
    crooped_image=image[1400:1750,500:1600]
    return crooped_image

#cropping the table of numbers from 0 to 20
def cropping_note(image):
  crooped_image=image[800:1400,0:2481]
  return crooped_image

#detecting all the empty squares for numbers after comma
def detect_comma(name_image,name_template):
  img = cv2.imread(name_image)
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_gray=cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  template= cv2.imread(name_template, 0)    
  image=cropping_comma(img_gray)
  #la fonction bilateralFilter permet d'éliminer le bruit sur l'image 
#   image=cv2.bilateralFilter(image , 11,17,17)
#   template=cv2.bilateralFilter(template,11,17,17)

  w ,h=template.shape #w: width , h: height le tuple (w,h) : correspond à la taille du carreau
  
  #print((w,h))
  #cv2.TM_CCOEFF_NORMED : c'est une parmi les 6 méthodes qui serent à determiner l'objet en question
  #c est la meilleure dans ce contexte , est la plus précise
  res=cv2.matchTemplate(image,template,cv2.TM_CCOEFF_NORMED) 
  
  threshold=0.8
  #determination des coordonnées du template
  loc=np.where( res>= threshold) 
  point=list()
  if loc :
    for pt in zip(*loc[::-1]):  
      #print((pt[0]+w, pt[1]+h)) 
      point.append((pt[0]+w, pt[1]+h))

      cv2.imwrite('image2.png',cv2.rectangle(image ,pt , (pt[0]+w, pt[1]+h),(0 , 255 , 255) ,5))
  img3=image.copy()
  for i in point:
    img3[i[1]][i[0]]=120
  cv2.imwrite('point_gris.png',img3)
  x=np.where(image!=img3) 
  #print('\n\nle point est de l image img[237,793],est : \n',img3[237][793])
  cropping (img3, 250, 610 , 255 ,625,'cropped_25.png')
  return x #,point
def valeur_firgule(base_image,template_image):
  liste1=detect_comma(base_image,template_image)
  diffrence=list(zip(*liste1[::-1]))
  #print('\nla difference en forme de tuple :\n')
  #print(diffrence)
  copy=diffrence
  firgule=['0.00','0.25','0.50','0.75']
  #while (len(diffrence)>0):
  for i in diffrence:
    if (i[0]<460) and (i[0]>320):
      diffrence.remove(i)
      try:
        firgule.remove('0.00')
      except :
        '0.00' not in firgule
      #  print('la firgule n est pas 0')
    if (i[0]<625) and (i[0]>490):
      diffrence.remove(i)
      try:
        firgule.remove('0.25')
      except:
        '0.25' not in firgule
      #print('la firgule n est pas 0.25')
    if (i[0]<800) and (i[0]>650):
      diffrence.remove(i)
      #print('la firgule n est pas 0.5')
      try:
        firgule.remove('0.50')
      except:
        '0.50' not in firgule
    if (i[0]<960) and (i[0]>805) :
      diffrence.remove(i)
      try:
        firgule.remove('0.75')
      except :
        '0.75' not in firgule
      #print('la firgule n est pas 0.75')
  # if diffrence == [] :
  #   break   
  #print(diffrence)
  #print(firgule)
  return firgule
def detect_note_line1(name_image,name_template):
  img = cv2.imread(name_image)
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_gray=cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  template= cv2.imread(name_template, 0)    
  image=cropping_note(img_gray)
  #print(image.shape)
#   image=cv2.bilateralFilter(image , 11,17,17)
#   template=cv2.bilateralFilter(template,11,17,17)
  w ,h=template.shape
  #print((w,h))

  res=cv2.matchTemplate(image,template,cv2.TM_CCOEFF_NORMED)
  threshold=0.8
  loc=np.where( res>= threshold)
  point=list()
  #print(min(loc[0])+106,max(loc[0])+106)
  if loc :
    for pt in zip(*loc[::-1]):  
      #print((pt[0]+w, pt[1]+h)) 
      point.append((pt[0]+w, pt[1]+h))

      cv2.imwrite('image3.png',cv2.rectangle(image ,pt , (pt[0]+w, pt[1]+h),(0 , 255 , 255) ,5))
  img3=image.copy()
  for i in point:
    img3[i[1]][i[0]]=120
  cv2.imwrite('point_gris.png',img3)
  x=np.where(image!=img3) 
  #print('\n\nle point est de l image img[237,793],est : \n',img3[237][793])
  cropping (img3, 280, 2110  , 320 ,2130,'cropped_10.png')
  return x #,point
def valeur_note(base_image,template_image):
  liste1=detect_note_line1(base_image,template_image)
  diffrence=list(zip(*liste1[::-1]))
  #print('\nla difference en forme de tuple :\n')
  #
  #print(diffrence)
  lista= list()
  for i in diffrence :
    if (i[1]>200 and i[1]<230) :
        lista.append(i)

   

  print(sorted(lista))
  copy=diffrence
  note=['00','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
  #while (len(diffrence)>0):
  for i in diffrence:
    if (i[0]<490) and (i[0]>310) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('00')
      except :
        '00' not in note
        #print('la note n est pas 0')
    if (i[0]<650) and (i[0]>490) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('1')
      except:
        '1' not in note
      #print('la note n est pas 1')
    if (i[0]<830) and (i[0]>660) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      #print('la note n est pas 2',i)
      try:
        note.remove('2')
      except:
        '2' not in note
    if (i[0]<1000) and (i[0]>830) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('3')
      except :
        '3' not in note
      #print('la note n est pas 3')
    if (i[0]<1170) and (i[0]>1000) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('4')
      except :
        '4' not in note
      #print('la note n est pas 4')
    if (i[0]<1340) and (i[0]>1170) and (i[1]<320) and (i[1]>170) :
      diffrence.remove(i)
      try:
        note.remove('5')
      except :
        '5' not in note
      #print('la note n est pas 5')
    if (i[0]<1505) and (i[0]>1340) and (i[1]<320) and (i[1]>170) :
      diffrence.remove(i)
      try:
        note.remove('6')
      except :
        '6' not in note
      #print('la note n est pas 6')
    if (i[0]<1660) and (i[0]>1508) and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('7')
      except :
        '7' not in note
      #print('la note n est pas 7')
    if (i[0]<1830) and (i[0]>1660) and (i[1]<250) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('8')
      except :
        '8' not in note
      #print('la note n est pas 8')
    if (i[0]<2000) and (i[0]>1830)  and (i[1]<320) and (i[1]>170):
      diffrence.remove(i)
      try:
        note.remove('9')
      except :
        '9' not in note
      #print('la note n est pas 9')
    if (i[0]<2200) and (i[0]>2000) and (i[1]<320) and (i[1]>170) :
      diffrence.remove(i)
      try:
        note.remove('10')
      except :
        '10' not in note
      #print('la note n est pas 10')
    if (i[0]<490) and (i[0]>310) and (i[1]<580) and (i[1]>425):
      diffrence.remove(i)
      try:
        note.remove('11')
      except :
        '11' not in note
        #print('la note n est pas 11')
    if (i[0]<700) and (i[0]>490) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('12')
      except :
        '12' not in note
        #print('la note n est pas 12')
    if (i[0]<920) and (i[0]>700) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('13')
      except :
        '13' not in note
        #print('la note n est pas 13')
    if (i[0]<990) and (i[0]>920) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('14')
      except :
        '14' not in note
        #print('la note n est pas 14')
    if (i[0]<1150) and (i[0]>990) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('15')
      except :
        '15' not in note
        #print('la note n est pas 15')
    if (i[0]<1320) and (i[0]>1150) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('16')
      except :
        '16' not in note
        #print('la note n est pas 16')
    if (i[0]<1480) and (i[0]>1320) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('17')
      except :
        '17' not in note
        #print('la note n est pas 17')
      
    if (i[0]<1645) and (i[0]>1480) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('18')
      except :
        '18' not in note
        #print('la note n est pas 18')
    if (i[0]<1810) and (i[0]>1645) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('19')
      except :
        '19' not in note
        #print('la note n est pas 19')
    if (i[0]<2000) and (i[0]>1810) and (i[1]>425) and (i[1]<580):
      diffrence.remove(i)
      try:
        note.remove('20')
      except :
        '20' not in note
 
  return note
def note_etudiant(base_image,template_image):
      
  valeur = valeur_note(base_image,template_image)
  
  if (len(valeur)==1):
    virgule=float(valeur_firgule(base_image,template_image)[0])
    noti=float(valeur[0])
    note=virgule+noti
  else :
    print(len(valeur))   
  
  return valeur 
for i in range(11,13):
    image_name="/home/soulaimen/Desktop/stage_z_consulting/object_detection/template_matching/exam/image00{}.png".format(i)
    print(image_name)
    print("\nla note associer est : ",note_etudiant(image_name,'empty_template.png'))

  

In [ ]:
point

In [ ]:
img3=image.copy()
for i in point:
  img3[i[1]][i[0]]=120 

In [ ]:
cv2_imshow(img3)
cv2_imshow(image)

In [ ]:
from PIL import Image as im
def extract_text (image,i,min_x , min_y, max_x , max_y):
  I = image[min_x: max_x, min_y:max_y]
  data = im.fromarray(I)
  data.save('./images/image{}.png'.format(i))

In [ ]:
x=np.where(image!=img3)

In [ ]:
min_y=min(list(x[0]))
max_y=max(list(x[0]))
min_x=min(list(x[1]))
max_x=max(list(x[1]))

In [ ]:
from PIL import Image as im
def extract_text (image,min_x , min_y, max_x , max_y):
  I = image[min_y: max_y, min_x:max_x]
  data = im.fromarray(I)
  data.save('image.png')

In [ ]:
extract_text(img3,min_x , min_y, max_x , max_y)

In [ ]:
point